In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import differential_evolution, minimize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Data Loading
mix_data=pd.read_excel("/content/drive/MyDrive/Product_mix/Mixed_Budget.xlsx")


In [ ]:
num_agencies = len(mix_data["Agency"])
list1=mix_data["Budgted T/O"]
list2= mix_data["GP %"]


for i in range(len(list1)):
    if list1[i] < 0:
        list1[i] = 0


print("Count  :  ",num_agencies)

Count  :   69


In [ ]:
list1.sum()

4683763864.732279

In [ ]:
#DEFAULT VALUES

current_sales_total=np.sum(list1)

print("Current total Sales:", current_sales_total)
print("Current total profit:", np.sum(list1*list2))

Current total Sales: 4683763864.732279
Current total profit: 732384857.2936119


In [ ]:
#variable Calculation

length_sales= len(mix_data["Budgted T/O"])
average_sales=current_sales_total/len(mix_data["Budgted T/O"])


In [ ]:
#contribution
mix_data["contribution"] = mix_data["Budgted T/O"] / np.sum(mix_data["Budgted T/O"])
mix_data.head()

In [ ]:
list3 = mix_data["contribution"]

In [ ]:
def negative_profit(x):
    total_sales = np.sum(x)
    profit = -np.dot(x, list2)
    return profit

x0 = np.tile(average_sales, length_sales)

### DEFINE TARGET AND MARGINS

In [ ]:
#define target value

target_val = 4000000




In [ ]:
margin=[]

for i in range(len(list1)):
  if (list1[i] / current_sales_total) < 0.01:
    margin.append(0.0001)
  elif (list1[i] / current_sales_total) < 0.05:
    margin.append(0.0002)
  else:
    margin.append(0.0005)


In [ ]:
ones_list = [1 for _ in range(num_agencies)]
ml = [a - b for a, b in zip(ones_list, margin)]
mh = [a + b for a, b in zip(ones_list, margin)]

lower = [ ml* s * rate for ml,s in zip(ml,list1)]
higher = [ mh* s * rate for mh,s in zip(mh,list1)]

bounds =[(m,n) for m,n in zip(lower,higher)]


In [ ]:


options = {
    'ftol': 1e-6,  # Tolerance for the change in the objective function
    'gtol': 1e-6,  # Tolerance for the gradient (if applicable)
    'xtol': 1e-6,  # Tolerance for the step size
}

constraints = [({'type': 'eq', 'fun': lambda x: target_val - np.sum(x)}) ,({'type': 'ineq', 'fun': lambda x: x})]

result_global = differential_evolution(negative_profit, bounds)
result_mid = minimize(negative_profit, result_global.x, method='SLSQP', bounds=bounds, constraints=constraints, options=options)
result = minimize(negative_profit, result_mid.x, method='SLSQP', bounds=bounds, constraints=constraints, options=options)



<ipython-input-126-8af26dfca8ed>:12: OptimizeWarning: Unknown solver options: gtol, xtol
  result_mid = minimize(negative_profit, result_global.x, method='SLSQP', bounds=bounds, constraints=constraints, options=options)
<ipython-input-126-8af26dfca8ed>:13: OptimizeWarning: Unknown solver options: gtol, xtol
  result = minimize(negative_profit, result_mid.x, method='SLSQP', bounds=bounds, constraints=constraints, options=options)


In [ ]:
optimal_dif = -result_global.fun
print(optimal_dif)
optimal_mid = -result_mid.fun
print(optimal_mid)
optimal_fi = -result.fun
print(optimal_fi)

if optimal_dif >= optimal_mid and optimal_dif >= optimal_fi:
  mode = result_global
elif optimal_mid >=optimal_dif and optimal_mid>= optimal_fi:
  mode = result_mid
else:
  mode = result
print(-mode.fun)


732568106.3916582
732431341.683365
732431677.8555272
732568106.3916582


In [ ]:
optimal_sales = mode.x
optimal_profit = -mode.fun

print("Optimal Profit:", optimal_profit)
print("new value target: ", np.sum(optimal_sales))

mix_data["optimal_values"]= optimal_sales

Optimal Profit: 732568106.3916582
new value target:  4685167658.646836


In [ ]:
print(mix_data)

In [ ]:
mix_data.iloc[:,8:13]

,Budgted T/O,Budgeted GP,GP %,contribution,optimal_values
0,6.124742e+05,3.155045e+05,0.515131,0.000131,5.966693e+05
1,2.000000e+07,6.262704e+06,0.313135,0.004270,1.986993e+07
2,6.000000e+06,2.317200e+06,0.386200,0.001281,6.182125e+06
3,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00
4,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00
...,...,...,...,...,...
64,4.439130e+07,3.328281e+06,0.074976,0.009478,4.427597e+07
65,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00
66,1.487602e+07,4.027736e+06,0.270754,0.003176,1.510946e+07
67,1.786758e+07,3.573515e+06,0.200000,0.003815,1.725174e+07


# Iterative Process

In [ ]:
l=[(1,1),(0.999,1.001),(0.998,1.002),(0.997,1.003)]

In [ ]:
for a,b in l:
    bounds = [(a* s * rate, b* s * rate) for s in list1]

    constraints = [({'type': 'eq', 'fun': lambda x: target_val - np.sum(x)}) ,({'type': 'ineq', 'fun': lambda x: x})]

    result_global = differential_evolution(negative_profit, bounds)
    result_mid = minimize(negative_profit, result_global.x, method='SLSQP', bounds=bounds, constraints=constraints)
    result = minimize(negative_profit, result_mid.x, method='SLSQP', bounds=bounds, constraints=constraints)

    optimal_dif = -result_global.fun
    #print(optimal_dif)
    optimal_mid = -result_mid.fun
    #print(optimal_mid)
    optimal_fi = -result.fun
    #print(optimal_fi)

    if optimal_dif >= optimal_mid and optimal_dif >= optimal_fi:
      mode = result_global
    elif optimal_mid >=optimal_dif and optimal_mid>= optimal_fi:
      mode = result_mid
    else:
      mode = result
    #print(-mode.fun)

    optimal_sales = mode.x
    optimal_profit = -mode.fun


    #print("Optimal Sales Values:", optimal_sales)
    print("Optimal Profit:", optimal_profit)

Optimal Profit: 732384857.2936118
Optimal Profit: 732913473.5914195
Optimal Profit: 733047491.1426114
Optimal Profit: 733465546.7592878
